Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.923120
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.791807
Minibatch accuracy: 43.8%
Validation accuracy: 53.3%
Minibatch loss at step 100: 0.643845
Minibatch accuracy: 75.0%
Validation accuracy: 69.4%
Minibatch loss at step 150: 0.391582
Minibatch accuracy: 93.8%
Validation accuracy: 74.7%
Minibatch loss at step 200: 1.288442
Minibatch accuracy: 68.8%
Validation accuracy: 77.1%
Minibatch loss at step 250: 0.983623
Minibatch accuracy: 75.0%
Validation accuracy: 74.9%
Minibatch loss at step 300: 0.095744
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 350: 0.697383
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.832098
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 450: 0.462627
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
Minibatch loss at step 500: 0.668802
Minibatch accuracy: 68.8%
Validation accuracy: 81.2%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [13]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    max_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(max_pool + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    max_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(max_pool + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.447987
Minibatch accuracy: 12.5%
Validation accuracy: 10.7%
Minibatch loss at step 50: 2.028787
Minibatch accuracy: 18.8%
Validation accuracy: 34.2%
Minibatch loss at step 100: 0.988032
Minibatch accuracy: 62.5%
Validation accuracy: 61.9%
Minibatch loss at step 150: 0.478372
Minibatch accuracy: 75.0%
Validation accuracy: 72.0%
Minibatch loss at step 200: 1.542313
Minibatch accuracy: 68.8%
Validation accuracy: 75.7%
Minibatch loss at step 250: 1.258481
Minibatch accuracy: 62.5%
Validation accuracy: 75.1%
Minibatch loss at step 300: 0.180890
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 350: 0.759960
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.623873
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.425694
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Minibatch loss at step 500: 0.603531
Minibatch accuracy: 75.0%
Validation accuracy: 81.3%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [148]:
batch_size = 128+32
patch_size = 5

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # 5,5,1,10
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, 10], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros(10))

  # 5,5,10,20
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, 10, 20], stddev=0.1))
  layer2_biases = tf.Variable(tf.zeros(20))
  
  # Fully connected
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * 20, 200], stddev=0.1))
  layer3_biases = tf.Variable(tf.zeros(200))

  layer5_w = tf.Variable(tf.truncated_normal([200, 64], stddev=0.1))
  layer5_b = tf.Variable(tf.zeros(64))

  # Output layer
  layer4_weights = tf.Variable(tf.truncated_normal(
      [64, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.zeros(num_labels))
    

  
  # Model.
  def model(data):
    # Input 28,28,1, Output 28,28,10
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    # Input 28,28,6 Output : 14,14,10
    max_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(max_pool + layer1_biases)
    hidden = tf.nn.dropout(hidden, 0.3)
    
    # Input 14,14,10 Output : 10,10,20 (since no padding is used)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    # Input 10,10,20 Output: 5,5,20
    max_pool = tf.nn.max_pool(conv, [1,2,2,1],[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(max_pool + layer2_biases)
    hidden = tf.nn.dropout(hidden, 0.3)
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden = tf.nn.relu(tf.matmul(hidden, layer5_w) + layer5_b)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [154]:
epochs=60
num_examples = train_dataset.shape[0]
from sklearn.utils import shuffle
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized, number epochs : {}, batch size : {}'.format(epochs, batch_size))
  for i in range(epochs):
    train_dataset, train_labels = shuffle(train_dataset, train_labels)
    total_accuracy = 0
    print('Epoch {} ...'.format(i+1))
    for offset in range(0, num_examples, batch_size):
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        total_accuracy += (accuracy(predictions, batch_labels) * len(batch_data))
    
    print('Train acc: {}'.format(total_accuracy / train_dataset.shape[0]))
    
    print('Val acc: {}'.format(accuracy(valid_prediction.eval(), valid_labels)))
  print('Test acc: {}'.format(accuracy(test_prediction.eval(), test_labels)))

Initialized, number epochs : 60, batch size : 160
Epoch 1 ...
Train acc: 75.9165
Val acc: 83.07
Epoch 2 ...
Train acc: 83.8645
Val acc: 84.7
Epoch 3 ...
Train acc: 85.0865
Val acc: 85.13
Epoch 4 ...
Train acc: 85.8725
Val acc: 86.11
Epoch 5 ...
Train acc: 86.2705
Val acc: 86.44
Epoch 6 ...
Train acc: 86.775
Val acc: 86.75
Epoch 7 ...
Train acc: 87.0625
Val acc: 87.12
Epoch 8 ...
Train acc: 87.269
Val acc: 87.03
Epoch 9 ...
Train acc: 87.46
Val acc: 87.11
Epoch 10 ...
Train acc: 87.5935
Val acc: 87.49
Epoch 11 ...
Train acc: 87.772
Val acc: 87.93
Epoch 12 ...
Train acc: 87.8845
Val acc: 87.55
Epoch 13 ...
Train acc: 88.051
Val acc: 87.68
Epoch 14 ...
Train acc: 88.0495
Val acc: 87.72
Epoch 15 ...
Train acc: 88.203
Val acc: 87.8
Epoch 16 ...
Train acc: 88.2855
Val acc: 87.56
Epoch 17 ...
Train acc: 88.395
Val acc: 88.03
Epoch 18 ...
Train acc: 88.416
Val acc: 88.26
Epoch 19 ...
Train acc: 88.5065
Val acc: 88.64
Epoch 20 ...
Train acc: 88.525
Val acc: 88.18
Epoch 21 ...
Train acc: 88.6305